In [1]:
import pandas as pd
import numpy as np
import time
import ee
import pickle
ee.Initialize()
from datetime import datetime
# df = pd.read_csv("combined_regular_clean_with_ssurgo_variables.csv")
df = pd.read_pickle("cwr_nwpr_dataset")
# df = df.loc[:10]
df.head()

,jurisdiction_type,da_number,latitude,longitude,cwa_determination,Index
14324,CWR,SPL-2015-00294,32.56200,-117.02018,1,1
14339,CWR,NAO-2019-02292,38.92073,-77.14066,1,2
14346,CWR,NAO-2019-00266,38.84361,-77.07721,1,3
14348,CWR,LRC-2003-22401,41.99860,-87.92959,0,4
14378,CWR,MVP-2019-00125,47.38801,-93.43970,0,5


In [2]:
def square(lat=45.475649, lon=-69.471018 , size=100):
  crs_proj = "EPSG:4326"  
  return ee.Geometry.Point([lon, lat], proj=crs_proj).buffer(size).bounds()

In [3]:
(ee.FeatureCollection("users/madhukarreddy/US_county_map")
 .filterBounds(square())
 .getInfo()
 .get("features")[0]
 .get('properties'))
#  .get('NAME'))

{'AFFGEOID': '0500000US23021',
 'ALAND': 10259018201,
 'AWATER': 1080044975,
 'COUNTYFP': '021',
 'COUNTYNS': '00581296',
 'GEOID': '23021',
 'LSAD': '06',
 'NAME': 'Piscataquis',
 'STATEFP': '23'}

In [4]:
df.head()

,jurisdiction_type,da_number,latitude,longitude,cwa_determination,Index
14324,CWR,SPL-2015-00294,32.56200,-117.02018,1,1
14339,CWR,NAO-2019-02292,38.92073,-77.14066,1,2
14346,CWR,NAO-2019-00266,38.84361,-77.07721,1,3
14348,CWR,LRC-2003-22401,41.99860,-87.92959,0,4
14378,CWR,MVP-2019-00125,47.38801,-93.43970,0,5


In [5]:
def find_county(lat=42.85821, lon=-76.70773, size=100):
    try:
        result = (ee.FeatureCollection("users/madhukarreddy/US_county_map")
         .filterBounds(square(lat, lon, size))
         .getInfo()
         .get("features")[0]
         .get('properties')
         .get('NAME'))
    except Exception as e:
        print(e)
        
        return np.nan
    
    return result
find_county()

'Cayuga'

In [6]:
def find_state(lat=42.85821, lon=-76.70773, size=100):
    try:
        result = (ee.FeatureCollection("users/madhukarreddy/US_county_map")
         .filterBounds(square(lat, lon, size))
         .getInfo()
         .get("features")[0]
         .get('properties')
         .get('STATEFP'))
    except:
        return np.nan
    
    return result
find_state()

'36'

In [7]:
print(datetime.now())

2021-04-01 19:35:44.120549


In [8]:
df["county"] = df.apply(lambda x: find_county(x.latitude, x.longitude), axis=1)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [9]:
print(datetime.now())

2021-04-01 19:48:10.818103


In [10]:
df["state"] = df.apply(lambda x: find_state(x.latitude, x.longitude), axis=1)

In [11]:
print(datetime.now())

2021-04-01 20:00:35.480739


In [12]:
np.mean(np.array(df.county.isna()))
df[df.county.isna()]

,jurisdiction_type,da_number,latitude,longitude,cwa_determination,Index,county,state
25537,CWR,SPN-2017-00207,37.88244,-122.31368,1,977,NaN,NaN
26644,CWR,LRE-2006-16087,43.26709,-82.52604,1,1068,NaN,NaN
68435,CWR,SWG-2015-00608,28.05945,-95.87867,1,1220,NaN,NaN
68892,CWR,SWG-2014-00656,28.76322,-95.62898,1,1296,NaN,NaN
68906,CWR,NAB-2015-01209,38.31433,-76.01150,1,1305,NaN,NaN
1409,NWPR,POA-2020-00528,56.96278,-132.92444,0,1641,NaN,NaN
4321,NWPR,POA-2020-00389,59.60128,-151.41705,0,2582,NaN,NaN
4322,NWPR,POA-2020-00388,59.60102,-151.41769,0,2583,NaN,NaN


In [13]:
np.mean(np.array(df.county.isna()))

0.002486016159105034

In [14]:
import pickle

# use protocol 3 for backwards compatibility with Python 3.6 on AWS
pickle.dump(df, open("2021.04.01_cwr_nwpr_us_county_raw0","wb"), protocol=3)
# df.to_csv("2021.03.17_us_county_raw0.csv")

# Read the file

In [15]:

df = pd.read_pickle("2021.04.01_cwr_nwpr_us_county_raw0")

In [16]:
df.head()

,jurisdiction_type,da_number,latitude,longitude,cwa_determination,Index,county,state
14324,CWR,SPL-2015-00294,32.56200,-117.02018,1,1,San Diego,06
14339,CWR,NAO-2019-02292,38.92073,-77.14066,1,2,Fairfax,51
14346,CWR,NAO-2019-00266,38.84361,-77.07721,1,3,Arlington,51
14348,CWR,LRC-2003-22401,41.99860,-87.92959,0,4,Cook,17
14378,CWR,MVP-2019-00125,47.38801,-93.43970,0,5,Itasca,27


In [17]:
np.mean(np.array(df.county.isna()))

0.002486016159105034

In [18]:
df[df.county.isna()].head()

,jurisdiction_type,da_number,latitude,longitude,cwa_determination,Index,county,state
25537,CWR,SPN-2017-00207,37.88244,-122.31368,1,977,NaN,NaN
26644,CWR,LRE-2006-16087,43.26709,-82.52604,1,1068,NaN,NaN
68435,CWR,SWG-2015-00608,28.05945,-95.87867,1,1220,NaN,NaN
68892,CWR,SWG-2014-00656,28.76322,-95.62898,1,1296,NaN,NaN
68906,CWR,NAB-2015-01209,38.31433,-76.01150,1,1305,NaN,NaN


In [19]:
df[df.county.isna()].shape

(8, 8)

In [20]:
df.loc[df.county.isna(),:].head(2)

,jurisdiction_type,da_number,latitude,longitude,cwa_determination,Index,county,state
25537,CWR,SPN-2017-00207,37.88244,-122.31368,1,977,NaN,NaN
26644,CWR,LRE-2006-16087,43.26709,-82.52604,1,1068,NaN,NaN


In [21]:
def find_county2(lat=43.26720, lon=-82.52610):
    try:
        return (ee.FeatureCollection("users/madhukarreddy/US_county_map")
         .filterBounds(square(lat, lon, size=10000))
         .getInfo()
         .get("features")[0]
         .get('properties')
         .get('NAME'))
    except:
        return np.nan

find_county2()

'Sanilac'

In [22]:
def find_state2(lat=43.26720, lon=-82.52610):
    try:
        return (ee.FeatureCollection("users/madhukarreddy/US_county_map")
         .filterBounds(square(lat, lon, size=10000))
         .getInfo()
         .get("features")[0]
         .get('properties')
         .get('STATEFP'))
    except:
        return np.nan

find_state2()

'26'

In [23]:
df_temp = df.loc[df.county.isna(),:].apply(lambda x: find_county2(x.latitude, x.longitude), axis=1)

In [24]:
np.mean(df_temp.isna())


0.125

In [25]:
df.loc[df.county.isna(),:][df_temp.isna()]

,jurisdiction_type,da_number,latitude,longitude,cwa_determination,Index,county,state
68435,CWR,SWG-2015-00608,28.05945,-95.87867,1,1220,NaN,NaN


In [26]:
def find_county3(lat=43.26720, lon=-82.52610, value=1):
    if type(value) != str:
        try:
            return (ee.FeatureCollection("users/madhukarreddy/US_county_map")
             .filterBounds(square(lat, lon, size=10000))
             .getInfo()
             .get("features")[0]
             .get('properties')
             .get('NAME'))
        except:
            return np.nan
    else:
        return value
find_county3()

'Sanilac'

In [27]:
def find_state3(lat=43.26720, lon=-82.52610, value=1):
    if type(value) != str:
        try:
            return (ee.FeatureCollection("users/madhukarreddy/US_county_map")
             .filterBounds(square(lat, lon, size=10000))
             .getInfo()
             .get("features")[0]
             .get('properties')
             .get('NAME'))
        except:
            return np.nan
    else:
        return value
find_state3()

'Sanilac'

In [28]:
df.apply(lambda x: find_state3(x.latitude, x.longitude, x.state), axis=1)

14324    06
14339    51
14346    51
14348    17
14378    27
         ..
7258     55
7262     55
7266     55
7272     55
7273     27
Length: 3218, dtype: object

In [29]:
# df.apply(lambda x: find_county2(x.latitude, x.longitude) if np.isnan(x.county) else x.county, axis=1)

df["county"] = df.apply(lambda x: find_county3(x.latitude, x.longitude, x.county), axis=1)
df["state"] = df.apply(lambda x: find_state3(x.latitude, x.longitude, x.state), axis=1)

In [30]:
df.head()

,jurisdiction_type,da_number,latitude,longitude,cwa_determination,Index,county,state
14324,CWR,SPL-2015-00294,32.56200,-117.02018,1,1,San Diego,06
14339,CWR,NAO-2019-02292,38.92073,-77.14066,1,2,Fairfax,51
14346,CWR,NAO-2019-00266,38.84361,-77.07721,1,3,Arlington,51
14348,CWR,LRC-2003-22401,41.99860,-87.92959,0,4,Cook,17
14378,CWR,MVP-2019-00125,47.38801,-93.43970,0,5,Itasca,27


In [31]:
np.mean(np.array(df.county.isna()))

0.00031075201988812925

In [32]:
import pickle

# use protocol 3 for backwards compatibility with Python 3.6 on AWS
pickle.dump(df, open("2021.04.01_cwr_nwpr_us_county_raw","wb"), protocol=3)
# df.to_csv("2021.03.17_us_county_raw.csv")

In [33]:
df = pd.read_pickle("2021.04.01_cwr_nwpr_us_county_raw")
df.head()

,jurisdiction_type,da_number,latitude,longitude,cwa_determination,Index,county,state
14324,CWR,SPL-2015-00294,32.56200,-117.02018,1,1,San Diego,06
14339,CWR,NAO-2019-02292,38.92073,-77.14066,1,2,Fairfax,51
14346,CWR,NAO-2019-00266,38.84361,-77.07721,1,3,Arlington,51
14348,CWR,LRC-2003-22401,41.99860,-87.92959,0,4,Cook,17
14378,CWR,MVP-2019-00125,47.38801,-93.43970,0,5,Itasca,27


In [34]:
len(df.state.unique())
len(df.county.unique())
set(df.state.unique())
np.mean(df.state.isna())

0.00031075201988812925